In [37]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch

torch.set_printoptions(edgeitems=2, linewidth=75)
torch.manual_seed(123)

from torchvision import datasets
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # <1>
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True) # <2>

Files already downloaded and verified
Files already downloaded and verified


In [38]:
len(cifar10) , len(cifar10_val)

(50000, 10000)

In [39]:
img , label = cifar10[99]
img , label

(<PIL.Image.Image image mode=RGB size=32x32>, 1)

In [40]:
from torchvision import transforms
to_tensor = transforms.ToTensor() #텐서로 변환 
img_t = to_tensor(img)
img_t.shape , img_t.dtype

(torch.Size([3, 32, 32]), torch.float32)

In [41]:
# 채널별 평균 , 표준편차 
img_t.view(3,-1).mean(dim=1) # 3*1024로 변환시키고 평균(3은 dim=1) = dim=1은 두번째 차원을 의미하므로 -1에 대한 차원을 의미 따라서 3은 그대로 있음

tensor([0.4611, 0.2478, 0.1610])

In [42]:
img_t.view(3,-1).std(dim=1) # 3*1024로 변환시키고 표준편차

tensor([0.3045, 0.1606, 0.1530])

In [43]:
img , label = cifar10[99]
img_t = to_tensor(img)
img_t.shape

torch.Size([3, 32, 32])

In [44]:
img_t = img_t.view(-1).unsqueeze(0)

In [45]:
img_t.view(-1).unsqueeze(0).shape # 차원이랑 뉴런수가 맞아야하므로 

torch.Size([1, 3072])

In [46]:
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(3072,512),
    nn.Tanh(),
    nn.Linear(512,2),
    nn.Softmax(dim=1) # 두번쨰 차원에 대해 소프트 맥스 적용 
)

In [47]:
out = model(img_t)
out

tensor([[0.5117, 0.4883]], grad_fn=<SoftmaxBackward0>)

In [48]:
_ , index = torch.max(out,dim=1)
index # 이미지 식별 결과

tensor([0])

In [50]:
softmax = nn.Softmax(dim=1)
log_softmax = nn.LogSoftmax(dim=1)
x = torch.tensor([[0.0, 104.0]])
softmax(x) ,log_softmax(x)

(tensor([[0., 1.]]), tensor([[-104.,    0.]]))

In [51]:
torch.exp(log_softmax(x))

tensor([[0., 1.]])

In [54]:
label_map = {0: 0, 2: 1} # 두가지 분류만 가정 
cifar2 = [(img, label_map[label]) for img, label in cifar10 if label in [0, 2]]
cifar2_val = [(img, label_map[label]) for img, label in cifar10_val if label in [0, 2]]

In [61]:
model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

loss = nn.NLLLoss()
img, label = cifar2[0]
to_tensor = transforms.ToTensor() #텐서로 변환 
img_t = to_tensor(img)
out = model(img_t.view(-1).unsqueeze(0))

loss(out, torch.tensor([label]))

tensor(0.5856, grad_fn=<NllLossBackward0>)

In [62]:
img, label = cifar2[0]
img = to_tensor(img)
model(img.view(-1).unsqueeze(0))

tensor([[-0.8137, -0.5856]], grad_fn=<LogSoftmaxBackward0>)

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim

model = nn.Sequential(
            nn.Linear(3072, 512),
            nn.Tanh(),
            nn.Linear(512, 2),
            nn.LogSoftmax(dim=1))

learning_rate = 1e-2
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 10
for epoch in range(n_epochs):
    for img, label in cifar2:
        img = to_tensor(img)
        out = model(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label])) 
                
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 1.226961
Epoch: 1, Loss: 1.507641
Epoch: 2, Loss: 1.994812
Epoch: 3, Loss: 2.348707
Epoch: 4, Loss: 3.125479
Epoch: 5, Loss: 3.360244
Epoch: 6, Loss: 3.657878
Epoch: 7, Loss: 3.710430
Epoch: 8, Loss: 4.111611
Epoch: 9, Loss: 4.269182
